# Hyperparameter Optimization
Hyperparameters detail specifics of the learning process or model
that are not learned in the training process. For example, the 
number of random trees is a hyperparameter for a random forest.

The choice for hyperpramters strongly influence model performance,
so it is important to be able to optimize them as well. AMPL
offers a variety of hyperparameter optimization methods including
random sampling, grid search, and Bayesian optimization. Here
we demonstrate an optimization method using Bayesian optimization.

## JSON Settings.
- `'hyperparam':True` This setting indicates that we are performing
a hyper parameter search instead of just training one model.
- `'search_type':'hyperopt'` This specifies the hyperparameter
search method. Other options include grid, random, and geometric.
Specifications for each hyperparameter search method is different,
please refer to the full documentation.
- `'model_type':'RF|10'` This means AMPL will try 10 times to 
find the best set of hyperparameters using random forests. In 
production this parameter could be set to 100 or more.
- `'rfe':'uniformint|8,512'` The Bayesian optimizer will uniformly
search between 8 and 512 for the best number of random forest estimators.
Similarly `rfd` stands for random forest depth and `rff` stands for
random forest features.
- `result_dir` Now expects two parameters. The first directory
will contain the best trained models while the second directory will
contain all models trained in the search.

In [1]:
import atomsci.ddm.utils.hyperparam_search_wrapper as hsw
import os

descriptor_type = 'rdkit_raw'
output_dir = 'output_kcna3_rdkit_raw'
tmp_dir = 'tmp_kcna3_rdkit_raw'
config = {
    "hyperparam": "True",
    "prediction_type": "regression",

    "dataset_key": "dataset/curated_kcna3_ic50.csv",
    "id_col": "compound_id",
    "smiles_col": "base_rdkit_smiles",
    "response_cols": "avg_pIC50",

    "splitter":"scaffold",
    "split_uuid": "c0313c63-8936-4297-925b-ee537b66dd89",
    "previously_split": "True",

    "featurizer": "computed_descriptors",
    "descriptor_type" : descriptor_type,
    "transformers": "True",

    "search_type": "hyperopt",
    "model_type": "RF|10",
    "rfe": "uniformint|8,512",
    "rfd": "uniformint|8,512",
    "rff": "uniformint|8,200",

    "result_dir": f"./{output_dir},./{tmp_dir}"
}


if not os.path.exists(f'./{output_dir}'):
    os.mkdir(f'./{output_dir}')

params = hsw.parse_params(config)
hs = hsw.build_search(params)
hs.run_search()


/usr/WS1/he6/AMPL_virtualenv_1.6/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/usr/WS1/he6/AMPL_virtualenv_1.6/lib/python3.9/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. jax requires jaxlib to be installed. See https://github.com/google/jax#installation for installation instructions.
2024-01-04 15:29:28,218 Created a dataset hash '389b161b7a4eb2304323a7dfddacfacc' fr

model_performance|train_r2|train_rms|valid_r2|valid_rms|test_r2|test_rms|model_params|model

rf_estimators: 153, rf_max_depth: 12, rf_max_feature: 136
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024-01-04 15:29:28,307 ['dataset_hash'] are not part of the accepted list of parameters and will be ignored
2024-01-04 15:29:28,333 Created a dataset hash '389b161b7a4eb2304323a7dfddacfacc' from dataset_key '/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/examples/tutorials2023/dataset/curated_kcna3_ic50.csv'


num_model_tasks is deprecated and its value is ignored.
RF model with computed_descriptors and rdkit_raw      
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2024-01-04 15:29:28,389 Reading descriptor spec table from /usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/data/descriptor_sets_sources_by_descr_type.csv
2024-01-04 15:29:28,400 Attempting to load featurized dataset
2024-01-04 15:29:28,440 Got dataset, attempting to extract data
2024-01-04 15:29:28,507 Creating deepchem dataset
2024-01-04 15:29:28,508 Using prefeaturized data; number of features = 200
2024-01-04 15:29:28,523 Previous dataset split restored
/usr/WS1/he6/AMPL_virtualenv_1.6/lib/python3.9/site-packages/deepchem/data/datasets.py:473: RuntimeWarning: overflow encountered in multiply
  X_m2 += dx * (X - X_means)

2024-01-04 15:29:28,539 Wrote transformers to ./tmp_kcna3_rdkit_raw/curated_kcna3_ic50/RF_computed_descriptors_scaffold_regression/e594fbaa-99b5-401b-a995-fcce5661c3c5/transformers.pkl
2024-01-04 15:29:28,541 Transforming response data
2024-01-04 15:29:28,541 Transforming feature data
/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarn

...

model_performance|0.976|0.283|0.722|0.754|0.812|0.736|332_173_175|./tmp_kcna3_rdkit_raw/curated_kcna3_ic50_model_a8364b53-2d19-452e-ae6f-97c917704c1b.tar.gz

rf_estimators: 460, rf_max_depth: 61, rf_max_feature: 158                        
 90%|█████████ | 9/10 [00:14<00:01,  1.59s/trial, best loss: 0.27398817546977505]

2024-01-04 15:29:42,546 ['dataset_hash'] are not part of the accepted list of parameters and will be ignored
2024-01-04 15:29:42,570 Created a dataset hash '389b161b7a4eb2304323a7dfddacfacc' from dataset_key '/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/examples/tutorials2023/dataset/curated_kcna3_ic50.csv'


num_model_tasks is deprecated and its value is ignored.                          
RF model with computed_descriptors and rdkit_raw                                 
 90%|█████████ | 9/10 [00:14<00:01,  1.59s/trial, best loss: 0.27398817546977505]

2024-01-04 15:29:42,602 Attempting to load featurized dataset
2024-01-04 15:29:42,651 Got dataset, attempting to extract data
2024-01-04 15:29:42,720 Creating deepchem dataset
2024-01-04 15:29:42,721 Using prefeaturized data; number of features = 200
2024-01-04 15:29:42,741 Previous dataset split restored
/usr/WS1/he6/AMPL_virtualenv_1.6/lib/python3.9/site-packages/deepchem/data/datasets.py:473: RuntimeWarning: overflow encountered in multiply
  X_m2 += dx * (X - X_means)

2024-01-04 15:29:42,755 Wrote transformers to ./tmp_kcna3_rdkit_raw/curated_kcna3_ic50/RF_computed_descriptors_scaffold_regression/ea14600a-90bc-4fbb-88b1-af8fc2204344/transformers.pkl
2024-01-04 15:29:42,756 Transforming response data
2024-01-04 15:29:42,757 Transforming feature data
/usr/WS1/he6/code/ATOM/AMPL1.6/atomsci/ddm/pipeline/transformations.py:255: RuntimeWarning: invalid value encountered in divide
  X = np.nan_to_num((X - self.X_means) * X_weight / self.X_stds)

2024-01-04 15:29:42,760 Transforming respo

model_performance|0.976|0.282|0.725|0.750|0.817|0.726|460_61_158|./tmp_kcna3_rdkit_raw/curated_kcna3_ic50_model_ea14600a-90bc-4fbb-88b1-af8fc2204344.tar.gz

100%|██████████| 10/10 [00:16<00:00,  1.64s/trial, best loss: 0.27398817546977505]
Generating the performance -- iteration table and Copy the best model tarball.
Best model: ./tmp_kcna3_rdkit_raw/curated_kcna3_ic50_model_c165d4d6-e0be-4be9-a6aa-301539c26fbb.tar.gz, valid R2: 0.726011824530225


The best model will be saved in `output_kcna3_rdkit_raw` along with a csv file
containing regression performance for all trained models.

The rest of the models are saved in `tmp_kcna3_rdkit_raw`. These models can be
explored using `get_filesystem_perf_results`.

In [3]:
import atomsci.ddm.pipeline.compare_models as cm

result_df = cm.get_filesystem_perf_results(
    result_dir='tmp_kcna3_rdkit_raw',
    pred_type='regression'
)

# sort by validation r2 score to find the best model
result_df = result_df.sort_values(by='best_valid_r2_score', ascending=False)

2024-01-04 15:44:04,987 Warning: column names have been changed to align with get_multitask_perf_from_tracker(): featurizer is now features and <subset>_<metric> has been changed to best_<subset>_<metric>.


Found data for 10 models under tmp_kcna3_rdkit_raw


The column `model_parameters_dict` contains hyperparamters used for the best model.

In [4]:
result_df.iloc[0].model_parameters_dict

'{"rf_estimators": 160, "rf_max_depth": 247, "rf_max_features": 103}'